In [11]:
import requests
import time
import pymysql
import os

from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint

In [2]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

### Getting list of repositories and authors from the file

In [3]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
    
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

2079


In [6]:
filename = 'scraped_data_mini.txt' # use full version later

In [10]:
paper_data = {}

with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            if len(line):
                current_repo = line[0]
                contributors = [line[1:]]         

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

### Creating graph

In [ ]:
def lookup_id(author,df):
        author_id=""
    #     for index, row in df.iterrows():
        for index, row in df.iterrows():
            #print(index)
            #print(k1,v2)
            if author==row["name"]:
                author_id=(df.at[index,"authorID"])
                break
        return author_id